In [1]:
from fastai.collab import *
from fastai.tabular import *

In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import lightgbm as lgb
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500
#import pandas_profiling

In [3]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

Train a model on all train data and use shape to get the feature contribution for each customers of each of the 200 features


Then we can use that contribution for the target in the collab filtering as the target and we train a model and then predict that for the test set

In [5]:
exclusion = ['ID_code', 'target'] 

feats = [c for c in train_df.columns if c not in exclusion]

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df[feats], train_df['target'], test_size=0.20, random_state=42)

parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31, #31
    'feature_fraction': 0.5,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'learning_rate': 0.05, #0.05
    'verbose': 30
    #'min_data_in_leaf': 200
}

trn_lgb = lgb.Dataset(X_train, y_train)
val_lgb = lgb.Dataset(X_test, y_test)
clf = lgb.train(parameters,
                 train_set=trn_lgb,
                 valid_sets=[trn_lgb, val_lgb],
                 num_boost_round=3000,
                 early_stopping_rounds=50,
                 verbose_eval=100)


Training until validation scores don't improve for 50 rounds.
[100]	training's auc: 0.897363	valid_1's auc: 0.854042
[200]	training's auc: 0.927577	valid_1's auc: 0.874363
[300]	training's auc: 0.943822	valid_1's auc: 0.882643
[400]	training's auc: 0.954541	valid_1's auc: 0.887822
[500]	training's auc: 0.962521	valid_1's auc: 0.889474
[600]	training's auc: 0.968652	valid_1's auc: 0.89056
[700]	training's auc: 0.974139	valid_1's auc: 0.891359
[800]	training's auc: 0.978762	valid_1's auc: 0.891811
Early stopping, best iteration is:
[808]	training's auc: 0.979067	valid_1's auc: 0.891937


ValueError: Columns must be same length as key

In [38]:
test_pred = pd.DataFrame(clf.predict(X_test[feats],pred_contrib=True))

In [39]:
test_pred['total'] = abs(test_pred.iloc[:,:-1]).sum(1)

test_pred.iloc[:,:200] = abs(test_pred.iloc[:,:200].div(test_pred['total'], axis=0))

In [41]:
X_test[[f'predict_contri_var_{i}' for i in range(200)]] = test_pred.iloc[:,:200]

In [ ]:
train_pred = pd.DataFrame(clf.predict(X_train[feats],pred_contrib=True))

In [ ]:
train_pred['total'] = abs(train_pred.iloc[:,:-1]).sum(1)

train_pred.iloc[:,:200] = abs(train_pred.iloc[:,:200].div(train_pred['total'], axis=0))

In [ ]:
X_train[[f'predict_contri_var_{i}' for i in range(200)]] = train_pred.iloc[:,:200]